In [1]:
import numpy as np
import math
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
import json
#
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score
from scipy.stats import truncnorm
#
import tensorflow as tf
# ignore the tensorflow warnings
tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import layers
#
import xgboost as xgb
#
#
import nevergrad as ng

#
from concurrent import futures
import time
import os
#
from scipy.optimize import Bounds
from scipy.optimize import minimize
#
import ast

2025-01-16 17:11:42.970245: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-16 17:11:42.972231: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-16 17:11:42.994989: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-16 17:11:43.028809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737018703.061331  377803 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737018703.07

## Setup

In [2]:
np.random.RandomState(1234)
np.random.seed(42)
n_list = [20,25]
m_list = [5, 10]
budget = {'default':20}#, (10,5):200, (15,5):150, (15,10):100}
maxiter = 350
K = 50
constraint_tolerance = .01
opt_list = [ng.optimizers.NGOpt, ng.optimizers.NGOptRW]#, ng.optimizers.NelderMead, ng.optimizers.Cobyla]
filename = "rastrigin.json"

In [3]:
def rastrigin(x, n):
    """
    Rastrigin function
    Global minimum: f(x*) = 0 at x* = (0,...,0)
    Search domain: [-5.12, 5.12]^n
    """
    A = 10
    sum = 0.0
    for i in range(n):
        sum += x[i]**2 - A*np.cos(2*np.pi*x[i])
    return A*n + sum

def get_truncated_normal(mean=5, sd=1, low=0, upp=10, size=None):
    """normal distribution between 0 and 10"""
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd).rvs(size)

In [4]:
def get_random_constants(k:int, mean=5., sd=1., seed=42, low=0, upp=10):
    np.random.seed(seed)
    return get_truncated_normal(size=k, mean=mean, sd=sd, low=low, upp=upp)

def get_random_indices(n:int, seed:int=42):
    indices = np.arange(n)
    np.random.seed(seed)
    np.random.shuffle(indices)
    return indices

def fill_with_constants(x:np.array, constants:np.array, indices:np.array):
    #return np.append(x,constants)[indices]
    return np.take(np.append(x,constants), indices)

def fill_with_constants_tf(x, constants:np.array, indices:np.array):
    y = tf.concat([x,constants], axis=-1)
    return tf.gather(y, indices)

In [5]:
ann_models = {n: tf.keras.models.load_model(f"models/ANN_{n}.h5") for n in n_list}
xgb_models = {n: xgb.Booster() for n in n_list}
for n in n_list:
    xgb_models[n].load_model(f"models/XGB_{n}.json")
#
with open('models/metadata.json', 'r') as file:
    metadata = json.load(file)
#
print("R^2 values:\n")
for n in metadata.keys():
    print(f"n={n}")
    print(f"ANN: {round(metadata[n]['ANN_R2'],4)}\nXGB: {round(metadata[n]['XGB_R2'],4)}")
    print("\n")
print(ann_models)
#print(xgb_models)

2025-01-16 17:12:00.134708: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


R^2 values:

n=20
ANN: 0.6602
XGB: 0.9114


n=25
ANN: 0.6605
XGB: 0.8846


{20: <Sequential name=sequential_4, built=True>, 25: <Sequential name=sequential_5, built=True>}


## Models and objective functions

In [6]:
def relu(x):
    return np.maximum(0, x)


def get_functions(n, constants, indices):
    #
    weights = ann_models[n].get_weights()
    def f_ann(x):
        x1 = fill_with_constants(x, constants, indices)
        z1 = np.dot(x1, weights[0]) + weights[1]
        a1 = relu(z1)
        z2 = np.dot(a1, weights[2]) + weights[3]
        a2 = relu(z2)
        z3 = np.dot(a2, weights[4]) + weights[5]
        return z3
    #
    #grad_f_ann = grad(f_ann)
    #
    def f_ann_tf(x):
        x1 = fill_with_constants_tf(x, constants, indices)
        x2 = tf.convert_to_tensor([x1])
        x2 = tf.cast(x2,dtype=tf.float32)
        return ann_models[n](x2)[0]
    #
    def grad_f_ann_tf(x):
        with tf.GradientTape() as tape:
            tape.watch(x)
            t2 = f_ann_tf(x)
            y = tape.jacobian(t2, x)
            return y
        
    def hess_tf(x):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            t2 = f_ann_tf(x)
            y = tape.gradient(t2, x)
            h = tape.jacobian(y, x)
            return y, h
    #
    def f_xgb(x):
        x1 = fill_with_constants(x, constants, indices)
        y = xgb_models[n].predict(xgb.DMatrix([x1]))#x1.reshape(1,-1))
        return y.item()
    #
    def f_rastrigin(x):
        x1 = fill_with_constants(x, constants, indices)
        return rastrigin(x1, n)
    #
    return f_rastrigin, f_ann_tf, grad_f_ann_tf, f_xgb, hess_tf

## Optimization methods

In [7]:
# optimization with scikit-opt (generic algorithm, particle swarm, and simulated annealing)\
from sko.GA import GA
from sko.PSO import PSO
from sko.SA import SAFast
#
def ga(m, f, size_pop, max_iter, prob_mut, constraint_c):
    constraint_eq = (
        lambda x: np.sum(x) - constraint_c,
    )
    lb, ub = [0.0]*m, [10.0]*m
    ga = GA(func=f, n_dim=m, size_pop=size_pop, max_iter=max_iter, prob_mut=prob_mut, lb=lb, ub=ub, constraint_eq=constraint_eq)
    start = time.time()
    best_x, _ = ga.run()
    end = time.time()
    return best_x, _, end-start
#
def pso(m, f, pop, max_iter, constraint_c):
    constraint_eq = (
        lambda x: np.sum(x) - constraint_c,
    )
    lb, ub = [0.0]*m, [10.0]*m
    pso = PSO(func=f, dim=m, pop=pop, max_iter=max_iter, lb=lb, ub=ub, constraint_eq=constraint_eq)
    start = time.time()
    best_x, _ = pso.run()
    end = time.time()
    return best_x, _, end-start

#
def sa(m, f, x0, T_max, T_min, L, max_stay_counter, constraint_c):
    constraint_eq = (
        lambda x: np.sum(x) - constraint_c,
    )
    lb, ub = [0.0]*m, [10.0]*m
    sa = SAFast(func=f, x0=x0, T_max=T_max, T_min=T_min, L=L, max_stay_counter=max_stay_counter, constraint_eq=constraint_eq, lb=lb, ub=ub)
    start = time.time()
    best_x, _ = sa.run()
    end = time.time()
    return best_x, _, end-start

In [8]:
def optimize_ng(m, f, constraint_c, optclass, budget=1_000, x_start=None, constraint_tolerance=.1):
    instrum = ng.p.Instrumentation(
        ng.p.Array(shape=(m,)).set_bounds(lower=0.0, upper=10),
    )
    optimizer = optclass(parametrization=instrum, budget=budget, num_workers=1)
    optimizer.parametrization.register_cheap_constraint(lambda x:np.sum(x[0]) - constraint_c)
    optname = str(optclass).split(".")[-1].split("'")[0]
    filepath = f"opti_result/{optname}_logs.txt"
    optimizer.register_callback("tell", ng.callbacks.ParametersLogger(filepath, append=False))
    #
    start = time.time()
    recommendation = optimizer.minimize(f, verbosity=0, batch_mode=False)
    #with futures.ThreadPoolExecutor(max_workers=optimizer.num_workers) as executor:
    #    recommendation = optimizer.minimize(f, verbosity=0, executor=executor, batch_mode=False)
    end = time.time()
    elapsed_time = end - start
    #
    with open(filepath, 'r') as file:
        lines = file.readlines()
        log = [ast.literal_eval(line) for line in lines]
    #
    ## select argument with minimum function value satisfying the constraint
    x_list = [l['0'] for l in log]
    x_constrained_list = [x for x in x_list if np.abs((np.sum(x)-constraint_c))/constraint_c < constraint_tolerance]
    #x_constrained_list = x_constrained_list or x_list
    if not x_constrained_list:
        x_constrained_list = x_list
        print("Could not satisfy constraints.")
    loss_array = np.array([f(x) for x in x_constrained_list])
    min_index = loss_array.argmin()
    x_min = x_constrained_list[min_index]
    #
    #return np.asarray(recommendation.value)[0][0], elapsed_time
    return np.asarray(x_min), elapsed_time


def optimize_grad(m, f, grad_f, constraint_c, maxiter=maxiter, x_start=None):
    if x_start is None: x_start = np.ones(m)
    eq_cons = {'type': 'eq',
               'fun' : lambda x:np.sum(x)-constraint_c
               }
    bounds = Bounds([0.0]*m, [10.0]*m)
    start = time.time()
    result = minimize(fun=f, x0=x_start, jac=grad_f,constraints=[eq_cons], method='SLSQP',
    options={'ftol': 1e-12, 'disp': False, 'maxiter':maxiter}, bounds=bounds)
    end = time.time()
    #
    elapsed_time = end - start
    return result.x, elapsed_time

In [9]:
results = []
for n in n_list:
    m_list_current = [m for m in m_list if m < n]
    for m in m_list_current:
        for k in range(K):
            print(f"Optimizing for n={n} and m={m}, trial {k+1}/{K}")
            #
            seed = k + 1_000*n + 1_000_000*m
            results_current = {'n': n, 'm': m, 'k': k}
            constants = get_random_constants(n-m, seed=seed)
            indices = get_random_indices(n, seed=seed)
            constraint_c = 5*m
            current_budget = budget.get((n,m), budget['default'])
            #
            f_rastrigin, f_ann, grad_f_ann,f_xgb, hess_tf = get_functions(n, constants, indices)
            f_ann, grad_f_ann = tf.function(f_ann), tf.function(grad_f_ann)
            #
            x_min = {}
            for optclass in opt_list:
                optname = 'XGB_' + str(optclass).split(".")[-1].split("'")[0]
                results_current[optname] = {}
                x_min, elapsed_time = optimize_ng(m, f_xgb, constraint_c, optclass=optclass, budget=current_budget, constraint_tolerance=constraint_tolerance, x_start=None)
                y_min = f_rastrigin(x_min)
                results_current[optname]['x_min'] = x_min.tolist()
                results_current[optname]['time'] = elapsed_time
                results_current[optname]['loss'] = y_min
                results_current[optname]['budget'] = current_budget
            
            # generic algorithm
            x_min_ga, _, elapsed_time_ga = ga(m, f_xgb, size_pop=50, max_iter=50, prob_mut=0.1, constraint_c=constraint_c)
            y_min_ga = f_rastrigin(x_min_ga)
            results_current['XGB_GA'] = {'loss': y_min_ga, 'time': elapsed_time_ga, 'x_min': x_min_ga.tolist()}
            
            # pso
            x_min_pso, _, elapsed_time_pso = pso(m, f_xgb, pop=50, max_iter=50, constraint_c=constraint_c)
            y_min_pso = f_rastrigin(x_min_pso)
            results_current['XGB_PSO'] = {'loss': y_min_pso, 'time': elapsed_time_pso, 'x_min': x_min_pso.tolist()}
            
            # simulated annealing
            x0 = np.ones(m)
            x_min_sa, _, elapsed_time_sa = sa(m, f_xgb, x0, T_max=50, T_min=1, L=10, max_stay_counter=10, constraint_c=constraint_c)
            y_min_sa = f_rastrigin(x_min_sa)
            results_current['XGB_SA'] = {'loss': y_min_sa, 'time': elapsed_time_sa, 'x_min': x_min_sa.tolist()}
            
            # xgboost + SLSQP
            x_min_xgb_slsqp, elapsed_time_xgb_slsqp = optimize_grad(m, f_xgb, grad_f_ann, constraint_c)
            y_min_xgb_slsqp = f_rastrigin(x_min_xgb_slsqp)
            results_current['XGB_ANN_SLSQP'] = {'loss': y_min_xgb_slsqp, 'time': elapsed_time_xgb_slsqp, 'x_min': x_min_xgb_slsqp.tolist()}
            
            
            # SLSQP + ann
            x_min_ann, elapsed_time_ann = optimize_grad(m, f_ann, grad_f_ann, constraint_c)
            #
            y_min_ann = f_rastrigin(x_min_ann)
            #
            results_current['ANN_ANN_SLSQP'] = {'loss': y_min_ann, 'time': elapsed_time_ann, 'x_min': x_min_ann.tolist()}
            
            
            results.append(results_current)

            #
            with open(os.path.join('opti_result', filename), 'w') as file:
                json.dump(results, file)

Optimizing for n=20 and m=5, trial 1/50


/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/parametrization/core.py:348: UserWarning: Lambda as constraint is not advised because it may not be picklable.
  warnings.warn("Lambda as constraint is not advised because it may not be picklable.")
/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/optimization/recaster.py:272: FinishedUnderlyingOptimizerWarning: Underlying optimizer has already converged, returning random points
  warnings.warn(
/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/parametrization/core.py:348: UserWarning: Lambda as constraint is not advised because it may not be picklable.
  warnings.warn("Lambda as constraint is not advised because it may not be picklable.")
/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/optimization/recaster.py:272: FinishedUnderlyingOptimizerWarning: Underlying optimizer has already converged, returning random points
  warnings.warn(


Optimizing for n=20 and m=5, trial 2/50


/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/parametrization/core.py:348: UserWarning: Lambda as constraint is not advised because it may not be picklable.
  warnings.warn("Lambda as constraint is not advised because it may not be picklable.")
/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/optimization/recaster.py:272: FinishedUnderlyingOptimizerWarning: Underlying optimizer has already converged, returning random points
  warnings.warn(


Could not satisfy constraints.


...

Optimizing for n=25 and m=10, trial 49/50


/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/parametrization/core.py:348: UserWarning: Lambda as constraint is not advised because it may not be picklable.
  warnings.warn("Lambda as constraint is not advised because it may not be picklable.")
/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/optimization/recaster.py:272: FinishedUnderlyingOptimizerWarning: Underlying optimizer has already converged, returning random points
  warnings.warn(
/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/parametrization/core.py:348: UserWarning: Lambda as constraint is not advised because it may not be picklable.
  warnings.warn("Lambda as constraint is not advised because it may not be picklable.")
/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/optimization/recaster.py:272: FinishedUnderlyingOptimizerWarning: Underlying optimizer has already converged, returning random points
  warnings.warn(


Could not satisfy constraints.
Optimizing for n=25 and m=10, trial 50/50


/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/parametrization/core.py:348: UserWarning: Lambda as constraint is not advised because it may not be picklable.
  warnings.warn("Lambda as constraint is not advised because it may not be picklable.")
/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/optimization/recaster.py:272: FinishedUnderlyingOptimizerWarning: Underlying optimizer has already converged, returning random points
  warnings.warn(
/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/parametrization/core.py:348: UserWarning: Lambda as constraint is not advised because it may not be picklable.
  warnings.warn("Lambda as constraint is not advised because it may not be picklable.")
/home/cican/miniconda3/envs/ml/lib/python3.12/site-packages/nevergrad/optimization/recaster.py:272: FinishedUnderlyingOptimizerWarning: Underlying optimizer has already converged, returning random points
  warnings.warn(


Could not satisfy constraints.
